In [1]:
!pip install fastai==2.7.1
!pip install timm>=0.6.2.dev0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/213.6 kB ? eta -:--:--

     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/213.6 kB 1.4 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 92.2/213.6 kB 943.7 kB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 1.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.1 MB/s eta 0:00:00


...

    Uninstalling fastcore-1.5.6:


      Successfully uninstalled fastcore-1.5.6


  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.7


    Uninstalling fastai-2.7.7:


      Successfully uninstalled fastai-2.7.7


In [ ]:
from fastai.vision.all import *
import timm

In [ ]:
train_path = '../input/paddy-disease-classification/train_images'
test_path = '../input/paddy-disease-classification/test_images'

In [ ]:
test_files = get_image_files(test_path).sorted()

In [ ]:
# Load training Data
dls = ImageDataLoaders.from_folder(train_path, valid_pct=0.1, item_tfms=Resize(640, 480), batch_tfms=aug_transforms(size=224, min_scale=0.75), bs=64//2)

In [ ]:
learn = vision_learner(dls, 'beit_base_patch16_224_in22k', metrics=error_rate, cbs=GradientAccumulation(64), path='.').to_fp16()

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
learn.fine_tune(60, 0.003)

In [ ]:
preds,_ = learn.tta(dl=dls.test_dl(test_files), n=3)
#Learner.tta (ds_idx=1, dl=None, n=4, item_tfms=None, batch_tfms=None, beta=0.25, use_max=False)
#The final predictions are (1-beta) multiplied by this average + beta multiplied by the predictions obtained with the transforms of the dataset. Set beta to None to get a tuple of the predictions and tta results. You can also use the maximum of all predictions instead of an average by setting use_max=True.

In [ ]:
idxs = preds.argmax(dim=1)

In [ ]:
vocab = np.array(dls.vocab)
results = pd.Series(vocab[idxs], name="idxs")

In [ ]:
submit = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
submit['label'] = results
submit.to_csv('beit_base_patch16_224_in22k_60_003.csv', index=False)